In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import winsound
import random

In [3]:
df= pd.read_csv("../../data/processed/oline_merge.csv")

In [4]:
df = df[df.year >= df.draft_yr+4]
df =df[df.year != 2020]
len(df)

201

In [5]:
df = df.drop('year',axis=1)
df = df.drop('playername',axis=1)
df = df.drop('team',axis=1)
df = df.drop('basesalarycap (10^8)',axis=1)
df = df.drop('cashspent (M)',axis=1)
df = df.drop('caphit (M)',axis=1)
df = df.drop('Unnamed: 0',axis=1)
df = df.drop('Unnamed: 0.1',axis=1)
df = df.drop('catchpercent',axis=1)
df = df.drop('draft_yr',axis=1)

df = df.drop('yardspergame_run',axis=1)
df = df.drop('yardsperatt',axis=1)
df = df.drop('yardspertarget',axis=1)
df = df.drop('yardsperrec',axis=1)
df = df.drop('recpergame',axis=1)
df = df.drop('yardspergame_rec',axis=1)


In [6]:
df

,age,draft_pos,attempts,yards_run,tds_run,longgain_run,Percenthit (%),g,gs,tgt,rec,yards_rec,tds_rec,firstdowns,longgain_rec,fumbles,team_adjusted_line_yards,team_running_back_yards,team_stuffed_rate
5,34,7,211,898,5,32,0.945802,15,15,23,17,142,0,7,22,3,18.0,15.0,17.0
6,33,7,251,1042,7,90,0.355530,16,16,26,20,208,1,9,52,3,26.0,23.0,28.0
7,31,7,37,72,0,13,7.728473,3,3,6,3,8,0,0,7,1,30.0,32.0,26.0
8,30,7,327,1485,11,80,10.748185,16,16,36,30,222,0,9,49,7,10.0,4.0,12.0
9,29,7,21,75,0,17,9.268023,1,1,3,2,18,0,1,9,0,13.0,15.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,30,-1,133,548,4,30,0.540993,16,1,26,22,200,1,11,50,0,1.0,9.0,2.0
560,29,-1,96,375,4,26,0.216154,8,3,13,12,129,0,7,27,0,13.0,18.0,14.0
562,25,10,223,857,12,25,4.888417,15,15,49,31,207,2,8,23,3,19.0,26.0,26.0
575,32,-1,11,23,0,6,1.195122,16,7,20,11,47,1,3,12,1,32.0,32.0,32.0


modelling

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [9]:
X = df.drop('Percenthit (%)',axis=1).values
y = df['Percenthit (%)'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [1]:
esv = []
# while True: 
for i in range(5):
    
    X_train.shape
    model = Sequential()

    model.add(Dense(18, activation="relu"))
    model.add(Dense(13, activation="relu"))
    model.add(Dense(13, activation="relu"))
    

    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")
    model.fit(x=X_train, y= y_train, validation_data=(X_test,y_test),batch_size=32,epochs=50)
    
    predictions = model.predict(X_test)
    esv.append(explained_variance_score(y_test,predictions))
    print('\n'+'\n'+"ESV: " + str(explained_variance_score(y_test,predictions))+ '\n'+'\n'+'\n')
    if(explained_variance_score(y_test,predictions) > .30):
        winsound.Beep(1047, 250)
        winsound.Beep(1047, 250)
        break
#     winsound.Beep(1397,250)

esv.sort(reverse=True)
print("\n")
print("MAX: " + str(max(esv)))
print("\n")
print(esv)
winsound.Beep(784, 125)

NameError: name 'X_train' is not defined

In [45]:
#model.save('./savedmodels/norookiednn')

INFO:tensorflow:Assets written to: ./savedmodels/norookiednn\assets


In [10]:
loaded_model = tf.keras.models.load_model('./savedmodels/norookiednn/')
predictions = loaded_model.predict(X_test)

print("MSE: ",mean_squared_error(y_test,predictions) )
print("RMSE: ",np.sqrt(mean_squared_error(y_test,predictions)) )
print("MAE: ",mean_absolute_error(y_test,predictions) )
print("ESV: ", explained_variance_score(y_test, predictions))

MSE:  3.335409621457654
RMSE:  1.8263103847532747
MAE:  1.3321803925332953
ESV:  0.4973884956852528


In [11]:
Aaron_Jones2020 = np.array([26,182,201,1104,9,77,14,14,63,47,355,2,15,30,2,5,3,1])
Aaron_Jones2020.shape
X_transformed = scaler.fit_transform(Aaron_Jones2020[:, np.newaxis])
print(X_transformed)
print(loaded_model.predict(X_transformed.T))

[[2.26654578e-02]
 [1.64097915e-01]
 [1.81323663e-01]
 [1.00000000e+00]
 [7.25294651e-03]
 [6.89029918e-02]
 [1.17860381e-02]
 [1.17860381e-02]
 [5.62103354e-02]
 [4.17044424e-02]
 [3.20942883e-01]
 [9.06618314e-04]
 [1.26926564e-02]
 [2.62919311e-02]
 [9.06618314e-04]
 [3.62647325e-03]
 [1.81323663e-03]
 [0.00000000e+00]]
[[1.2715582]]


In [12]:
Aaron_Jones2020 = np.array([26,182,201,1104,9,77,14,14,63,47,355,2,15,30,2,5,3,1])
Aaron_Jones2020.shape
X_transformed = scaler.transform(Aaron_Jones2020.reshape(1, -1))
print(X_transformed, '\n')
print(loaded_model.predict(X_transformed))

[[2.26654578e-02 1.64097915e-01 1.81323663e-01 1.00000000e+00
  7.25294651e-03 6.89029918e-02 1.17860381e-02 1.17860381e-02
  5.62103354e-02 4.17044424e-02 3.20942883e-01 9.06618314e-04
  1.26926564e-02 2.62919311e-02 9.06618314e-04 3.62647325e-03
  1.81323663e-03 0.00000000e+00]] 

[[1.2715582]]


In [13]:
KenyanDrake_2020 = np.array([27,73,239,955,10,69,15,13,31,25,137,0,5,18,3,30,19,25])
KenyanDrake_2020.shape
X_transformed = scaler.transform(KenyanDrake_2020.reshape(1, -1))
print(X_transformed, '\n')
print(loaded_model.predict(X_transformed))

[[ 0.02357208  0.06527652  0.21577516  0.86491387  0.00815956  0.06165005
   0.01269266  0.01087942  0.02719855  0.02175884  0.12330009 -0.00090662
   0.00362647  0.01541251  0.00181324  0.02629193  0.01631913  0.02175884]] 

[[1.5619557]]
